In [2]:
# load relevant packages, set some display parameters
%reset -f

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import time

from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, RepeatedKFold
from math import sqrt
from scipy import stats
from sklearn import preprocessing
from sklearn import ensemble
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, learning_curve
# import ifsg
import glob, os #I ADDED
from sklearn.svm import SVR, LinearSVR #I ADDED
from sklearn.tree import DecisionTreeRegressor #I ADDED
import pickle
import re
import random
import os
import warnings
import seaborn as sns
import gc
from sklearn.decomposition import PCA

warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)

#ifsg.__version__

C:\NewProg\Anaconda\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
# load data
path_dataset = 'W:\\Work\\Assense\\ML\\chars_file'
observation_filename = os.path.join(path_dataset,'updates_chars_file_04_03.csv')
observation_table = pd.read_csv(observation_filename)#,usecols=range(9))#, index_col=0,skiprows=[0,1,2])#,usecols=range(12))
set(observation_table.company_name)

{'Actiview', 'Cola', 'Deloitte', 'Dts', 'Jsol', 'Mitsubishi', 'Ntt'}

In [5]:
observation_table.replace({'Precision':'Thoroughness',
                     'Decisiveness Vs. Hesitant':'Decisiveness',
                     'Purposefulness vs. Hesitant':'Decisiveness',
                     'Monitor and review _ process and product Monitoring':'Self Monitoring',
                     'Monitor And Review | Process And Product Monitoring':'Self Monitoring',
                     'Monitor and review – process and product Monitoring':'Self Monitoring',
                     'Quality Vs. Quantity':'Quantity vs Quality',
                     'Time management':'Time Management',
                     '(stress tolerance) Reaction to stress and difficulty':'Stress Tolerance',
                     'Stress Tolerance | Reaction To Stress And Difficulty':'Stress Tolerance'},inplace=True)
observation_table.head(20)

,id,company_name,Purposefulness Vs. Hesitant,Time Management,Verbal reasoning,Collegial Behavior,Adaptability to Changes,Performance IQ,Emotional Understanding,Spatial reasoning,Processing Speed,Appropriate Behaviour Understanding,Cognitive ability total score,Goal orientation,Orderliness,Performance under stress,Thoroughness,Agreeableness,Memory,Verbal Reasoning,Attention: Coping with Distractions,Quantitative Reasoning,Processing speed,Intelligence Quotient,Methodicalness,Self-efficacy,Working memory,Quantitative reasoning,Performance under Stress,Performance IQ,Stress Tolerance,Empathy,Conscientiousness,Planning,Perceptual Organization,Self Monitoring,Monitoring,Coping with stress,Teamwork,Decisiveness,Adaptability to changes,Risk Taking,Precision,Quantity vs Quality
0,5beadc3169056d543c06c4e66,Actiview,5.0,5.000,NaN,NaN,NaN,5.000,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.00000,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.000000,5.0,NaN,5.0,NaN,5.000000,NaN,NaN,NaN,5.0,NaN
1,12345678,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5C45C837378D814141D7B22C,Actiview,NaN,NaN,NaN,NaN,NaN,6.000,NaN,6.0,6.0,NaN,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5b421df4ff36d1e2d5a70e77,Actiview,1.0,1.000,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.000000,5.0,NaN,3.0,NaN,3.000000,NaN,NaN,NaN,6.0,NaN
4,5bb3c16069ad6d50f0356ab7,Actiview,4.0,3.000,NaN,NaN,NaN,3.000,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.000000,3.0,NaN,4.0,NaN,4.000000,NaN,NaN,NaN,3.0,NaN
5,5bb3c16069ad6d50f0356qq7,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN
6,5bb3c16069ad6d50f0356qq7jghgj,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN
7,5bcee66c197fa961f4d3d407,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5beac3789a2a034e386583aa,Actiview,4.0,4.000,NaN,NaN,NaN,3.000,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.000000,2.0,NaN,4.0,NaN,2.000000,NaN,NaN,NaN,44.0,NaN
9,5beadc3169056d543c06c4e6,Actiview,4.0,3.000,NaN,NaN,NaN,5.000,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.000000,5.0,NaN,4.0,NaN,4.000000,NaN,NaN,NaN,3.0,NaN


In [126]:
# look at specific company
company  ='Ntt'
observation_table = observation_table[observation_table.company_name==company]
observation_table.head()

,id,company_name,Purposefulness Vs. Hesitant,Time Management,Verbal reasoning,Collegial Behavior,Adaptability to Changes,Performance IQ,Emotional Understanding,Spatial reasoning,Processing Speed,Appropriate Behaviour Understanding,Cognitive ability total score,Goal orientation,Orderliness,Performance under stress,Thoroughness,Agreeableness,Memory,Verbal Reasoning,Attention: Coping with Distractions,Quantitative Reasoning,Processing speed,Intelligence Quotient,Methodicalness,Self-efficacy,Working memory,Quantitative reasoning,Performance under Stress,Performance IQ,Stress Tolerance,Empathy,Conscientiousness,Planning,Perceptual Organization,Self Monitoring,Monitoring,Coping with stress,Teamwork,Decisiveness,Adaptability to changes,Risk Taking,Precision,Quantity vs Quality
634,5c1897b941a9ba23f05d7d66,Ntt,NaN,4.000000,NaN,NaN,NaN,5.5,NaN,4.0,5.3,NaN,4.9,4.600000,3.000000,NaN,3.800000,NaN,NaN,5.6,4.750000,-1.0,NaN,NaN,4.25,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,3.600000,NaN,4.200000,NaN,NaN,NaN,4.000000,4.0,3.400000,NaN,NaN
635,5c1897d841a9ba23f05d7d67,Ntt,NaN,4.500000,NaN,NaN,NaN,5.5,NaN,5.4,5.5,NaN,5.3,5.500000,1.500000,NaN,5.000000,NaN,NaN,5.3,4.000000,-1.0,NaN,NaN,5.00,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.000000,NaN,4.500000,NaN,NaN,NaN,5.000000,5.0,4.500000,NaN,NaN
636,5c19c20bcb2685556560b4ed,Ntt,NaN,4.666667,NaN,NaN,NaN,5.2,NaN,4.4,4.8,NaN,5.2,6.000000,4.000000,NaN,5.333333,NaN,NaN,3.8,5.666667,-1.0,NaN,NaN,6.00,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,4.333333,NaN,4.666667,NaN,NaN,NaN,5.000000,5.0,3.333333,NaN,NaN
637,5c19c240cb2685556560b4ee,Ntt,NaN,4.333333,NaN,NaN,NaN,5.3,NaN,4.8,4.9,NaN,5.7,4.333333,2.000000,NaN,5.666667,NaN,NaN,5.1,6.000000,-1.0,NaN,NaN,5.00,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.666667,NaN,4.333333,NaN,NaN,NaN,4.333333,6.0,2.000000,NaN,NaN
638,5c1ae8f1b60c77108de9c20a,Ntt,NaN,4.666667,NaN,NaN,NaN,4.0,NaN,3.7,4.2,NaN,3.9,5.666667,3.333333,NaN,2.333333,NaN,NaN,4.0,-1.000000,-1.0,NaN,NaN,4.00,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,4.333333,NaN,4.000000,NaN,NaN,NaN,5.000000,5.0,5.000000,NaN,NaN


In [7]:
observation_table = observation_table.dropna(axis=1,how = 'all')
observation_table.head(20)

,id,company_name,Purposefulness Vs. Hesitant,Time Management,Verbal reasoning,Collegial Behavior,Adaptability to Changes,Performance IQ,Emotional Understanding,Spatial reasoning,Processing Speed,Appropriate Behaviour Understanding,Cognitive ability total score,Goal orientation,Orderliness,Performance under stress,Thoroughness,Agreeableness,Memory,Verbal Reasoning,Attention: Coping with Distractions,Quantitative Reasoning,Processing speed,Intelligence Quotient,Methodicalness,Self-efficacy,Working memory,Quantitative reasoning,Performance under Stress,Performance IQ,Stress Tolerance,Empathy,Conscientiousness,Planning,Perceptual Organization,Self Monitoring,Monitoring,Coping with stress,Teamwork,Decisiveness,Adaptability to changes,Risk Taking,Precision,Quantity vs Quality
0,5beadc3169056d543c06c4e66,Actiview,5.0,5.000,NaN,NaN,NaN,5.000,NaN,NaN,NaN,NaN,NaN,NaN,5.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,5.00000,NaN,NaN,NaN,NaN,5.0,NaN,NaN,5.000000,5.0,NaN,5.0,NaN,5.000000,NaN,NaN,NaN,5.0,NaN
1,12345678,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5C45C837378D814141D7B22C,Actiview,NaN,NaN,NaN,NaN,NaN,6.000,NaN,6.0,6.0,NaN,6.000000,NaN,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5b421df4ff36d1e2d5a70e77,Actiview,1.0,1.000,NaN,NaN,NaN,1.000,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,3.000000,5.0,NaN,3.0,NaN,3.000000,NaN,NaN,NaN,6.0,NaN
4,5bb3c16069ad6d50f0356ab7,Actiview,4.0,3.000,NaN,NaN,NaN,3.000,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,4.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.000000,3.0,NaN,4.0,NaN,4.000000,NaN,NaN,NaN,3.0,NaN
5,5bb3c16069ad6d50f0356qq7,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN
6,5bb3c16069ad6d50f0356qq7jghgj,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.00000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,NaN
7,5bcee66c197fa961f4d3d407,Actiview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5beac3789a2a034e386583aa,Actiview,4.0,4.000,NaN,NaN,NaN,3.000,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,3.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.000000,2.0,NaN,4.0,NaN,2.000000,NaN,NaN,NaN,44.0,NaN
9,5beadc3169056d543c06c4e6,Actiview,4.0,3.000,NaN,NaN,NaN,5.000,NaN,NaN,NaN,NaN,NaN,NaN,4.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,4.00000,NaN,NaN,NaN,NaN,4.0,NaN,NaN,4.000000,5.0,NaN,4.0,NaN,4.000000,NaN,NaN,NaN,3.0,NaN


In [8]:
observation_table.to_excel('yam.xlsx')


In [114]:
#observation_table = observation_table[~(observation_table.company_name.str.contains('Actiview|Deloitte|Cola'))]


In [130]:
observation_table.loc['n'] = observation_table.count()
observation_table.loc['mean'] = observation_table.mean()
observation_table.loc['median'] = observation_table.median()
observation_table.loc['std'] = observation_table.std()


In [132]:
observation_table
observation_table.to_excel('ntt.xlsx')

In [115]:
a = []
for col in observation_table.columns[2:]:
    a.append([col, observation_table[col].count(), observation_table[col].mean(),
              observation_table[col].median(),observation_table[col].std()])
#     csv=[col, observation_table[col].count(), observation_table[col].mean(),
#           observation_table[col].median(),observation_table[col].std()]

df1 = pd.DataFrame(a, columns =['measure','n','mean','median','std'] )
df1.to_excel('tot.xlsx')